# Tweet Classification
#### 23518008 Hidayaturrahaman & 23518032 Henry Menori




Melakukan klasifikasi pada tweet kedalam class Keluhan, Respon, atau Bukan Keluhan/Respon.

NLP S2 IF-ITB 2018/2019



## Data Understanding

In [4]:
data.head()

ID                 Timestamp  \
0  678844958237822977  21 Dec 2015, 02:50:25 PM   
1  678845619444703232  21 Dec 2015, 02:53:02 PM   
2  678846848619024384  21 Dec 2015, 02:57:55 PM   
3  678847097047678977  21 Dec 2015, 02:58:54 PM   
4  678849085621714944  21 Dec 2015, 03:06:49 PM   

                                               Tweet Keluhan Respon  \
0  @EL_Atheos @ridwankamil ya mungkin karena pere...     NaN    NaN   
1  @ridwankamil @dbmpkotabdg kang teman saya tert...      Ya    NaN   
2  Di tribun jabar biasanya suka di post agenda k...     NaN    NaN   
3  @dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...      Ya    NaN   
4  @diskamtam bapak/ibu mau tanya, kalo pemelihar...     NaN    NaN   

  Bukan Keluhan/Respon        Topik Umum               Topik Spesifik  \
0                   Ya     Bukan Keluhan                            -   
1                  NaN  Lingkungan Hidup                pohon tumbang   
2                   Ya     Bukan Keluhan                            -   
3                  NaN     Infrastruktur  lampu penerangan jalan umum   
4                   Ya     Bukan Keluhan                            -   

                                   Lokasi Waktu   
0                                       -      -  
1  Jalan Sangkuriang depan Polsek Coblong      -  
2                                       -      -  
3                Jalan depan Kampus LPKIA      -  
4                                       -      -

In [3]:
keluhan = 0
respon = 0
other = 0

for data in dataY:
    if data[0]==1: 
        keluhan += 1
    elif data[1]==1:
        respon += 1
    else:
        other += 1
    
print(" Total Data:{}, keluhan:{}, respon:{}, other:{}".format(dataY.shape[0], keluhan, respon, other))

 Total Data:7527, keluhan:2744, respon:917, other:3866


## Preprocessing Data

In [2]:
### ======== Preprocessing Data ===============================

import numpy as np
import pandas as pd

data = pd.read_csv('data.csv', sep=';')

dataClean = data[data.columns[2:6]]

dataX = dataClean[dataClean.columns[0]].values
dataY = dataClean[dataClean.columns[1:]].values

# ==== one hot encode data Y ========

for i in range(len(dataY)):
    for j in range(len(dataY[i])):
        if dataY[i][j] == 'Ya':
            dataY[i][j] = 1
        else:
            dataY[i][j] = 0
            
# ==== tokenization data X ==========

for i in range(len(dataX)):
    dataX[i] = dataX[i].split()
    
import re

# cleaning from symbol, link, and emoticon

for i in range(len(dataX)):
    for j in range(len(dataX[i])):
        
        dataX[i][j] = re.sub("\W", " ", dataX[i][j])
        dataX[i][j] = re.sub("_", " ", dataX[i][j])
        
        #coba hapus semua yang ada angkanyaaa
        
        if re.search(r"\bhttp\w+", dataX[i][j]) != None:
            dataX[i][j] = ""
        
        if re.search(r"\d", dataX[i][j]) != None:
            dataX[i][j] = ""
        
        
            
# Concating for normalizing word purpose

for i in range(len(dataX)):
    dataX[i] = ' '.join(dataX[i])
    dataX[i] = dataX[i].split()
    dataX[i] = ' '.join(dataX[i])
    

## Normalize word using prosa.ai API

In [95]:
# ====== Normalize word using prosa.ai API =============

import requests
import json

url = "https://api.prosa.ai/v1/normals"

# payload = "{\n  \"text\": \"EL Atheos ridwankamil ya mungkin karena perempuan bj merah yg paling kelihatan karena berada di depan laki2 jg sering jadi bahan joke RK#ridwankamil dbmpkotabdg kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun#Di tribun jabar biasanya suka di post agenda kang emil Tapi gak setiap hari sih RT Ry ory Klo saya k Bdg bs ketemu Kang ridwankamil dmn#dbmpkotabdg RT fajriattack Lapor pak ridwankamil lampu penerangan jalan depan kampus lpkia ngaplek#diskamtam bapak ibu mau tanya kalo pemeliharaan taman2 yg banyak dibagun skr gimana nantinya#Pa ridwankamil tempat kerja saya tgl 25 tetep masuk gimana itu aturannya pa kalo misalnya ga masuk dipotong gaji#pdamtirtawening ridwankamil 00106700466 punten sudah seminggu air PDAM tidak mengalir Mohon bantuannya segera#RT KendiAdiputra Anak SMA yang ketahuan mencuri helem di bip hari ini pak ridwankamil#pdamtirtawening ternyata sudah ada warga yg konfirm ke pdam katanya air tidak akan mengalir selama 1 bulan krn ada perbaikan ridwankamil#Kang ridwankamil jalan di Muararajeun Baru baru sebulanan dibenerin udah rusak lagi Yang ngerjain kaya ga serius#RT AdamMaulud Didinyamah kieu bosss Kumaha pak ridwankamil daerah sapharuhai yeuh kang parkir liar seeur pisan atuhda#Pak ridwankamil punten tolong marahin motor yg ga pake spakbor belakang Udah mah td kehujanan wajah rupawan sy kacepretan#dbmpkotabdg ridwankamil hatur nuhun dbmp lampu jl merdeka atos sae#Kalo PemkotBandung ridwankamil mau batasi kendaraan yg feasible dan bisa dilakukan segera dlm wewenang mrk berantas parkir badan jalan#RT iqbalfg Pak ridwankamil punten tolong marahin motor yg ga pake spakbor belakang Udah mah td kehujanan wajah rupawan sy kacepretan#Pak tadi saya habis dari Kebun Binatang itu kebersihannya masih kurang dan orangutannya juga jomblo pak mohon solusi pak ridwankamil#RT TadjiUINBdg ridwankamil lapor pak mobil ni 2x buang sampah ke pinggir jalan sepanjang jalan sersan bajuri no 24#Bonbin bandung agak kurang terawat kasian sama hewan hewannya Pak ridwankamil coba bonbin bandung di bikin kayak singapore zoo#Pak ridwankamil sy pnya saran angkot 08 dan Antapani yg Coklat dilebur sj tp arah trayeknya dibagi 2 kyk Cimbeuleuit dl#RT TadjiUINBdg ridwankamil lapor pak mobil ni 2x buang sampah ke pinggir jalan sepanjang jalan sersan bajuri no 24#ridwankamil pak sya lg mau pulang naek diangkot cheum ledeng ad ibu2 sma bpak2 ngmongin bpak tp tenang koq pak ngmongin nya yg baik2#dishub kotabdg sepertinya yang membuat jlnan macet adlah prmptan Hrsnya ada jambtn lyang ditiap perempatan ridwankamil#infobdg kalau macet begini di lihat di depannya karena apa macetnya angkot bus ngetem pasti ada sumber titik macetnya ridwankamil#SaurWargi reckythea tolg ad seorang wanita d jl eikman tiap pg jam 9 suka loncat k tngh jln ambil kunci mtr dgn tebusan 10rb Satpolppbdg#BimaAryaS DiskominfoBdg dishub jabar dishub kotabdg mendukung rencana badan hukum angkot jgn sampai kasus metromini tjd nanti di bgr#RT se bdg SaurWargi reckythea tolg ad seorang wanita d jl eikman tiap pg jam 9 suka loncat k tngh jln ambil kunci mtr dgn tebusan 10rb#EL Atheos hendronurkholis ridwankamil he he perempuan itu sebelumnya dh ditanya sama rk dah nikah pa masih single#Punten saya baru denger beritanya apa sudah ditertibkan modus mengambil kunci motor Satpolppbdg PRFMnews#Min dbmpkotabdg kalau dilihat dari foto di bawah ini sebenarnya Jl Cemara memiliki trotoar ga ya Tx#RT soowooBDG Min dbmpkotabdg kalau dilihat dari foto di bawah ini sebenarnya Jl Cemara memiliki trotoar ga ya Tx#ridwankamil Satpolppbdg Mohon diamankan Pa Ogah per4an Pasar Suci Cihaurgeulis membuat jalan Suci macet total amp kacau Nuhun PERHATIANA#ridwankamil dbmpkotabdg Mohon dipasang tiang utk LAMPU MERAH DI PER4AN PASAR SUCI CIHAURGEULIS SERING TERJADI MACET TOTAL IWGinting#Setuju ini sepanjang jl riau banyak yg parkir Malah ada yg di trotoar juga ridwankamil#pjudbmpbdg ass pak kiaracondong masih bnyk lampu jalan yg mati bbk sari 1mohon di perbaiki Kec Kircon Kel BabakanSari ridwankamil#Pak wali upami angkot di bandung kumaha nasibna Di jakarta metromini tos jadi korban ridwankamil#RT TadjiUINBdg ridwankamil lapor pak mobil ni 2x buang sampah ke pinggir jalan sepanjang jalan sersan bajuri no 24\"\n}"
text = ' '.join(chars[400:450])
print(text)
payload = json.dumps({'text': text})
headers = {
    'Content-Type': "application/json",
    'x-api-key': "shOC0xYA3hoZG6SZ50M2eucwiH1jyFBlzltY3kSD"
#     'Postman-Token': "f6cd5852-6186-4e19-b700-5a0e79ba4d41"
    }

response = requests.request("POST", url, data=payload, headers=headers)

normal = json.loads(response.text)
charsNormal = normal['text'].split(' ')

print(len(charsNormal))
print(charsNormal)

adiraksa adit aditsap77488878 adiwiyata adji adjie adl adlah adlh admin administrasi adminnya admnya adrianmpriyatna adsastrawidjaja adu aduan aduh aduhh adukan adukeun aduuh advent ady ae aendiri aer afansobirin affan afiat afrika afrikaa after aga agak agama agar agenda ageng ageung ageungan agkt agm agma agr agung agus agustus ah ahaaaaay
50
['adiraksa', 'adit', 'aditsap77488878', 'adiwiyata', 'adji', 'adjie', 'adalah', 'adlah', 'adalah', 'admin', 'administrasi', 'adminnya', 'adalahnya', 'adrianmpriyatna', 'adsastrawidjaja', 'adu', 'aduan', 'aduh', 'aduh', 'adukan', 'adukeun', 'aduuh', 'advent', 'ady', 'ae', 'aendiri', 'aer', 'afansobirin', 'affan', 'afiat', 'afrika', 'afrikaa', 'after', 'agak', 'agak', 'agama', 'agar', 'agenda', 'ageng', 'ageung', 'ageungan', 'angkot', 'agm', 'agma', 'agr', 'agung', 'agus', 'agustus', 'ah', 'alay']


In [21]:
normal = json.loads(response.text)
charsNormal = normal['text'].split(' ')

print(len(charsNormal))
print(charsNormal[1])

NameError: name 'json' is not defined

In [2]:
print(dataX[1])
print(dataY[0])

ridwankamil dbmpkotabdg kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun
[0 0 1]


## Prepare Data for trainin model

In [8]:
corpus = ' '.join(dataX).lower().split()

chars = sorted(list(set(corpus)))
char_to_int = dict((c,i) for i, c in enumerate(chars))
int_to_char = dict((i,c) for i, c in enumerate(chars))

import pickle 

pickle_out = open("dict.pickle","wb")
pickle.dump(char_to_int, pickle_out)
pickle_out.close()


n_chars = len(corpus)
n_vocab = len(chars)
print ("Total Words: ", n_chars)
print ("total Vocab: ", n_vocab)

Total Words:  121911
total Vocab:  10064


In [9]:
for i in range(len(dataX)):
    dataX[i] = [char_to_int[char] for char in dataX[i].lower().split()]

In [6]:
dataX.shape

(7527,)

## Train Model
- Naive Bayes
- Decission Tree
- Neural Net

In [11]:
#load data
data = numpy.load('embeddedWord.npz')
dataX = data['dataX']
dataY = data['dataY']

In [10]:
# Prepare data

dataY_svm = []
for i in range(len(dataY)):
    dataY_svm.append(numpy.argmax(dataY[i]))

In [50]:
# ==============     SVM    ======================



# Train model
from sklearn.model_selection import cross_val_score
from sklearn import svm

model = svm.SVC()
score = cross_val_score(model, dataX, dataY_svm, scoring='accuracy', cv=5 )
print(score, numpy.mean(score))

# model.fit(dataX, dataY_svm)

[0.9800929  0.96480744 0.96212625 0.95282392 0.78058511] 0.928087121843171


In [51]:
# =============== Naive Bayes ===================

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
score = cross_val_score(model, dataX, dataY_svm, scoring='accuracy', cv=5)
print(score, numpy.mean(score))

[0.29794293 0.29415671 0.23388704 0.26445183 0.25731383] 0.2695504679411722


In [52]:
# =============== Random Forest ===================

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
score = cross_val_score(model, dataX, dataY_svm, scoring='accuracy', cv=5)
print(score, numpy.mean(score))

[0.89847379 0.90438247 0.86245847 0.9089701  0.74800532] 0.8644580299363535


## Save model 

In [20]:
#================== pakai yang ini ========================
#maxlen 32; batch 16; fc 250  do-0.6 150 


import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.models import Model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
print(max_words)
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(n_vocab, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dropout(0.6))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

# get embedding layer
layer_name = 'flatten_4'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_train)


model_json = intermediate_layer_model.to_json()
with open("model_embedding.json", "w") as json_file:
	json_file.write(model_json)
# serialize weights to HDF5
intermediate_layer_model.save_weights("model_embedding.h5")
print("Saved model to disk")


numpy.savez('embeddedWord.npz', dataX = intermediate_output, dataY = dataY)
print("data saved!")

31
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 31, 32)            322048    
_________________________________________________________________
flatten_5 (Flatten)          (None, 992)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 250)               248250    
_________________________________________________________________
dropout_5 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               25100     
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 303       
Total params: 595,701
Trainable params: 595,701
Non-trainable params: 0
_________________________________________________________________
N

/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6021 samples, validate on 1506 samples
Epoch 1/10
6021/6021 [==============================] - 3s 502us/step - loss: 0.5577 - acc: 0.6788 - val_loss: 0.4833 - val_acc: 0.7253
Epoch 2/10
6021/6021 [==============================] - 2s 314us/step - loss: 0.4542 - acc: 0.7526 - val_loss: 0.4164 - val_acc: 0.7629
Epoch 3/10
6021/6021 [==============================] - 2s 359us/step - loss: 0.3745 - acc: 0.8149 - val_loss: 0.3773 - val_acc: 0.8021
Epoch 4/10
6021/6021 [==============================] - 2s 298us/step - loss: 0.2736 - acc: 0.8783 - val_loss: 0.3318 - val_acc: 0.8663
Epoch 5/10
6021/6021 [==============================] - 2s 347us/step - loss: 0.2045 - acc: 0.9152 - val_loss: 0.3462 - val_acc: 0.8590
Epoch 6/10
6021/6021 [==============================] - 2s 353us/step - loss: 0.1434 - acc: 0.9440 - val_loss: 0.3904 - val_acc: 0.8251
Epoch 7/10
6021/6021 [==============================] - 2s 312us/step - loss: 0.1077 - acc: 0.9585 - val_loss: 0.3665 - val_acc: 0.8712


ValueError: No such layer: flatten_4

In [51]:
from tensorflow.keras.models import model_from_json

tweet = "@ridwankamil Ya allah masih ada yang ngiri sama kang emil dan kota bandung nya 😂😂😂"

tweet = tweet.split()

# cleaning
import re
for i in range(len(tweet)):
    tweet[i] = re.sub("\W", " ", tweet[i])
    tweet[i] = re.sub("_", " ", tweet[i])
    if re.search(r"\bhttp\w+", tweet[i]) != None:
        tweet[i] = ""
    if re.search(r"\d", tweet[i]) != None:
        tweet[i] = ""
tweet = ' '.join(tweet)
tweet = tweet.split()
tweet = ' '.join(tweet) 

# convert to number
import pickle
from tensorflow.keras.preprocessing import sequence
import numpy as np
dict_file = open("dict.pickle", "rb")
char_to_int = pickle.load(dict_file)
tweet = [char_to_int[char] for char in tweet.lower().split()]
tweet = np.asarray(mytrain)
tweet = tweet.reshape(1,tweet.shape[0])
tweet = sequence.pad_sequences(tweet, maxlen=31)
# tweet = tweet.reshape(1,31)

# one hot vector
json_file = open( 'model_embedding.json' ,  'r' )
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_embedding.h5")
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
tweet = loaded_model.predict(tweet)

print(np.asarray(tweet[0]).shape)

1
(992,)


In [19]:
#maxlen 32; batch 16; fc 250  do-0.6 150 


import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Dense(250, activation= 'relu', input_shape=X_train[0].shape ))
model.add(Dropout(0.6))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'rmsprop' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=16, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 250)               248250    
_________________________________________________________________
dropout_4 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 100)               25100     
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 303       
Total params: 273,653
Trainable params: 273,653
Non-trainable params: 0
_________________________________________________________________
None
Train on 6021 samples, validate on 1506 samples
Epoch 1/10
6021/6021 [==============================] - 3s 459us/step - loss: 0.1848 - acc: 0.9234 - val_loss: 0.5704 - val_acc: 0.8251
Epoch 2/10
6021/6021 [==============================] - 2s 331us/step - loss: 0

In [18]:
dataX.shape

(7527, 992)

In [20]:
from tensorflow.keras.models import model_from_json

# ======= saving model ===============

model_json = model.to_json()
with open("model.json", "w") as json_file:
	json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# later...

# load json and create model

json_file = open( 'model.json' ,  'r' )
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Saved model to disk
Loaded model from disk
acc: 95.56%


In [37]:
#load data
import numpy
data = numpy.load('embeddedWord.npz')
dataX = data['dataX']
dataY = data['dataY']

In [45]:
X = data['dataX'][0]
X

array([ 1.57106608e-01,  4.76610273e-01,  2.32316226e-01,  4.58666086e-02,
        1.62675694e-01,  1.35985643e-01,  4.52228755e-01,  5.36736012e-01,
        1.07958548e-01,  4.31194872e-01,  5.50123215e-01,  1.03685632e-01,
        3.98033887e-01,  6.73777759e-02,  2.08472311e-01,  4.60713953e-01,
        5.06911397e-01,  2.25148261e-01,  2.46028930e-01,  4.60814079e-03,
        6.63964637e-03,  4.21138346e-01,  4.58641797e-01, -3.73029453e-03,
        5.48258051e-02,  4.73444223e-01,  2.37630501e-01,  1.64919913e-01,
        1.93461820e-01,  2.14828879e-01,  7.15295896e-02,  3.21918339e-01,
        1.57106608e-01,  4.76610273e-01,  2.32316226e-01,  4.58666086e-02,
        1.62675694e-01,  1.35985643e-01,  4.52228755e-01,  5.36736012e-01,
        1.07958548e-01,  4.31194872e-01,  5.50123215e-01,  1.03685632e-01,
        3.98033887e-01,  6.73777759e-02,  2.08472311e-01,  4.60713953e-01,
        5.06911397e-01,  2.25148261e-01,  2.46028930e-01,  4.60814079e-03,
        6.63964637e-03,  

In [9]:
# Prepare data

dataY_svm = []
for i in range(len(dataY)):
    dataY_svm.append(numpy.argmax(dataY[i]))

In [10]:
# ==============     SVM    ======================



# Train model
from sklearn.model_selection import cross_val_score
from sklearn import svm

model = svm.SVC()
# score = cross_val_score(model, dataX, dataY_svm, scoring='accuracy', cv=5)
# print(score, numpy.mean(score))

model.fit(dataX, dataY_svm)

import pickle



model_filename = "model_svm.pkl"  
with open(model_filename, 'wb') as file:  
    pickle.dump(model, file)

# Load from file
with open(model_filename, 'rb') as file:  
    model = pickle.load(file)

score = model.score(dataX, dataY_svm)  
print("Test score: {0:.2f} %".format(100 * score))  

Test score: 96.01 %


In [11]:
# =============== Naive Bayes ===================

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(dataX, dataY_svm)

import pickle



model_filename = "model_naive.pkl"  
with open(model_filename, 'wb') as file:  
    pickle.dump(model, file)

# Load from file
with open(model_filename, 'rb') as file:  
    model = pickle.load(file)

score = model.score(dataX, dataY_svm)  
print("Test score: {0:.2f} %".format(100 * score))  

Test score: 25.99 %


In [12]:
# =============== Random Forest ===================

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)

model.fit(dataX, dataY_svm)

import pickle



model_filename = "model_RForest.pkl"  
with open(model_filename, 'wb') as file:  
    pickle.dump(model, file)

# Load from file
with open(model_filename, 'rb') as file:  
    model = pickle.load(file)

score = model.score(dataX, dataY_svm)  
print("Test score: {0:.2f} %".format(100 * score))  

Test score: 99.76 %


## Load model and Test

In [21]:
from tensorflow.keras.models import model_from_json

#load data
data = numpy.load('embeddedWord.npz')
dataX = data['dataX']
dataY = data['dataY']

# ============= ANN =====================

json_file = open( 'model.json' ,  'r' )
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("ANN Model loaded")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_train, y_train, verbose=0)
print("ANN %s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

# prep target for other model

dataY_svm = []
for i in range(len(dataY)):
    dataY_svm.append(numpy.argmax(dataY[i]))


# ============= Random Forest ==================

import pickle

with open("model_RForest.pkl", 'rb') as file:  
    model = pickle.load(file)
print("Random forest model loaded")
    
score = model.score(dataX, dataY_svm)  
print("Random Forest acc: {0:.2f} %".format(100 * score))



# ============= SVM ==================

import pickle

with open("model_svm.pkl", 'rb') as file:  
    model = pickle.load(file)
print("SVM model loaded")

score = model.score(dataX, dataY_svm)  
print("SMV acc: {0:.2f} %".format(100 * score))


ANN Model loaded
ANN acc: 95.56%
Random forest model loaded
Random Forest acc: 99.76 %
SVM model loaded
SMV acc: 96.01 %


# Generating one hot vector 

In [10]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.models import Model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
X_train = sequence.pad_sequences(X_train, maxlen=max_words)


# create the model
model = Sequential()
model.add(Embedding(n_vocab, 10, input_length=max_words))
model.add(Flatten())
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())

# get embedding layer
layer_name = 'flatten_2'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_train)

numpy.savez('embeddedWord.npz', dataX = intermediate_output, dataY = dataY)
print('Data saved !')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 31, 10)            100640    
_________________________________________________________________
flatten_2 (Flatten)          (None, 310)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 933       
Total params: 101,573
Trainable params: 101,573
Non-trainable params: 0
_________________________________________________________________
None
Data saved !


# Additional Experiment with Tensorflow

In [4]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(11000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dense(250, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=16, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           352000    
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 250)               4000250   
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 753       
Total params: 4,415,753
Trainable params: 4,415,753
Non-trainable params: 0
_________________________________________________________________
None


/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6021 samples, validate on 1506 samples
Epoch 1/10
6021/6021 [==============================] - 28s 5ms/step - loss: 2.1790 - acc: 0.6554 - val_loss: 0.5508 - val_acc: 0.6662
Epoch 2/10
6021/6021 [==============================] - 26s 4ms/step - loss: 0.5315 - acc: 0.6852 - val_loss: 0.4611 - val_acc: 0.8008
Epoch 3/10
6021/6021 [==============================] - 26s 4ms/step - loss: 0.4513 - acc: 0.7268 - val_loss: 0.4409 - val_acc: 0.7833
Epoch 4/10
6021/6021 [==============================] - 28s 5ms/step - loss: 0.3670 - acc: 0.8071 - val_loss: 0.4024 - val_acc: 0.7911
Epoch 5/10
6021/6021 [==============================] - 27s 4ms/step - loss: 0.2352 - acc: 0.9053 - val_loss: 0.9085 - val_acc: 0.6228
Epoch 6/10
6021/6021 [==============================] - 26s 4ms/step - loss: 0.1183 - acc: 0.9603 - val_loss: 0.6063 - val_acc: 0.7897
Epoch 7/10
6021/6021 [==============================] - 27s 4ms/step - loss: 0.0692 - acc: 0.9813 - val_loss: 0.6184 - val_acc: 0.7926
Epoch 8

In [5]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(11000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=64, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           352000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               4000250   
_________________________________________________________________
dense_4 (Dense)              (None, 100)               25100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 4,377,653
Trainable params: 4,377,653
Non-trainable params: 0
_________________________________________________________________
None


/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6021 samples, validate on 1506 samples
Epoch 1/10
6021/6021 [==============================] - 8s 1ms/step - loss: 1.3957 - acc: 0.6399 - val_loss: 1.3104 - val_acc: 0.5002
Epoch 2/10
6021/6021 [==============================] - 8s 1ms/step - loss: 1.1565 - acc: 0.6961 - val_loss: 0.9861 - val_acc: 0.5494
Epoch 3/10
6021/6021 [==============================] - 8s 1ms/step - loss: 1.1098 - acc: 0.7439 - val_loss: 0.7921 - val_acc: 0.7758
Epoch 4/10
6021/6021 [==============================] - 9s 1ms/step - loss: 1.0329 - acc: 0.8084 - val_loss: 0.7876 - val_acc: 0.7711
Epoch 5/10
6021/6021 [==============================] - 9s 1ms/step - loss: 0.9756 - acc: 0.8372 - val_loss: 0.7248 - val_acc: 0.6720
Epoch 6/10
6021/6021 [==============================] - 8s 1ms/step - loss: 0.3039 - acc: 0.8764 - val_loss: 0.4733 - val_acc: 0.7598
Epoch 7/10
6021/6021 [==============================] - 9s 2ms/step - loss: 0.2031 - acc: 0.9171 - val_loss: 0.4055 - val_acc: 0.7986
Epoch 8/10
602

In [111]:
#maxlen 32; batch 64; fc 250 100 


import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(11000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=64, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 31, 32)            352000    
_________________________________________________________________
flatten_17 (Flatten)         (None, 992)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 250)               248250    
_________________________________________________________________
dense_40 (Dense)             (None, 100)               25100     
_________________________________________________________________
dense_41 (Dense)             (None, 3)                 303       
Total params: 625,653
Trainable params: 625,653
Non-trainable params: 0
_________________________________________________________________
None


/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6021 samples, validate on 1506 samples
Epoch 1/10
6021/6021 [==============================] - 2s 395us/step - loss: 0.5150 - acc: 0.7175 - val_loss: 0.4836 - val_acc: 0.6815
Epoch 2/10
6021/6021 [==============================] - 1s 177us/step - loss: 0.3805 - acc: 0.8106 - val_loss: 0.4910 - val_acc: 0.6985
Epoch 3/10
6021/6021 [==============================] - 1s 190us/step - loss: 0.2700 - acc: 0.8802 - val_loss: 0.4420 - val_acc: 0.7652
Epoch 4/10
6021/6021 [==============================] - 1s 186us/step - loss: 0.1623 - acc: 0.9354 - val_loss: 0.3805 - val_acc: 0.8490
Epoch 5/10
6021/6021 [==============================] - 1s 203us/step - loss: 0.0973 - acc: 0.9647 - val_loss: 0.4541 - val_acc: 0.8012
Epoch 6/10
6021/6021 [==============================] - 1s 195us/step - loss: 0.0635 - acc: 0.9808 - val_loss: 0.5093 - val_acc: 0.8006
Epoch 7/10
6021/6021 [==============================] - 1s 218us/step - loss: 0.0413 - acc: 0.9877 - val_loss: 0.4910 - val_acc: 0.8063


In [112]:
#maxlen 32; batch 16; fc 250 100 


import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(11000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=16, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 31, 32)            352000    
_________________________________________________________________
flatten_18 (Flatten)         (None, 992)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 250)               248250    
_________________________________________________________________
dense_43 (Dense)             (None, 100)               25100     
_________________________________________________________________
dense_44 (Dense)             (None, 3)                 303       
Total params: 625,653
Trainable params: 625,653
Non-trainable params: 0
_________________________________________________________________
None


/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6021 samples, validate on 1506 samples
Epoch 1/10
6021/6021 [==============================] - 5s 820us/step - loss: 0.4617 - acc: 0.7414 - val_loss: 0.4450 - val_acc: 0.7081
Epoch 2/10
6021/6021 [==============================] - 4s 605us/step - loss: 0.2707 - acc: 0.8777 - val_loss: 0.6940 - val_acc: 0.6669
Epoch 3/10
6021/6021 [==============================] - 4s 619us/step - loss: 0.1303 - acc: 0.9490 - val_loss: 0.4487 - val_acc: 0.7441
Epoch 4/10
6021/6021 [==============================] - 4s 604us/step - loss: 0.0641 - acc: 0.9789 - val_loss: 0.3876 - val_acc: 0.8628
Epoch 5/10
6021/6021 [==============================] - 4s 597us/step - loss: 0.0377 - acc: 0.9885 - val_loss: 0.4579 - val_acc: 0.8178
Epoch 6/10
6021/6021 [==============================] - 4s 626us/step - loss: 0.0253 - acc: 0.9931 - val_loss: 0.4658 - val_acc: 0.8404
Epoch 7/10
6021/6021 [==============================] - 4s 621us/step - loss: 0.0216 - acc: 0.9937 - val_loss: 0.4854 - val_acc: 0.8632


In [5]:
#maxlen 32; batch 16; fc 250 250 


import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(11000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(150, activation= 'relu' ))
model.add(Dropout(0.6))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.5, batch_size=32, verbose=1)
# Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 31, 32)            352000    
_________________________________________________________________
flatten (Flatten)            (None, 992)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               248250    
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               25100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 625,653
Trainable params: 625,653
Non-trainable params: 0
_________________________________________________________________
None

/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3763 samples, validate on 3764 samples
Epoch 1/10
3763/3763 [==============================] - 2s 510us/step - loss: 0.5587 - acc: 0.6750 - val_loss: 0.5340 - val_acc: 0.7050
Epoch 2/10
3763/3763 [==============================] - 1s 367us/step - loss: 0.4816 - acc: 0.7283 - val_loss: 0.4872 - val_acc: 0.7419
Epoch 3/10
3763/3763 [==============================] - 1s 346us/step - loss: 0.4152 - acc: 0.7796 - val_loss: 0.4647 - val_acc: 0.7593
Epoch 4/10
3763/3763 [==============================] - 1s 352us/step - loss: 0.3487 - acc: 0.8317 - val_loss: 0.4751 - val_acc: 0.7642
Epoch 5/10
3763/3763 [==============================] - 1s 347us/step - loss: 0.2790 - acc: 0.8769 - val_loss: 0.4310 - val_acc: 0.7855
Epoch 6/10
3763/3763 [==============================] - 1s 353us/step - loss: 0.2067 - acc: 0.9135 - val_loss: 0.4984 - val_acc: 0.7683
Epoch 7/10
3763/3763 [==============================] - 1s 396us/step - loss: 0.1556 - acc: 0.9395 - val_loss: 0.4673 - val_acc: 0.8131


In [8]:
#maxlen 32; batch 32; fc 100 100 


import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
X_train = dataX
y_train = dataY

max_words = max([len(x) for x in dataX])
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model
model = Sequential()
model.add(Embedding(11000, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(100, activation= 'relu' ))
model.add(Dropout(0.6))
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, epochs=10, validation_split=0.5, batch_size=32, verbose=1)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 31, 32)            352000    
_________________________________________________________________
flatten (Flatten)            (None, 992)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               99300     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 461,703
Trainable params: 461,703
Non-trainable params: 0
_________________________________________________________________
None

/Users/dayatura/tensorflow3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3763 samples, validate on 3764 samples
Epoch 1/10
3763/3763 [==============================] - 2s 401us/step - loss: 0.5713 - acc: 0.6700 - val_loss: 0.5520 - val_acc: 0.6823
Epoch 2/10
3763/3763 [==============================] - 1s 265us/step - loss: 0.5151 - acc: 0.6956 - val_loss: 0.5201 - val_acc: 0.7453
Epoch 3/10
3763/3763 [==============================] - 1s 250us/step - loss: 0.4655 - acc: 0.7491 - val_loss: 0.4747 - val_acc: 0.7470
Epoch 4/10
3763/3763 [==============================] - 1s 258us/step - loss: 0.4160 - acc: 0.7850 - val_loss: 0.4801 - val_acc: 0.7703
Epoch 5/10
3763/3763 [==============================] - 1s 281us/step - loss: 0.3377 - acc: 0.8448 - val_loss: 0.4583 - val_acc: 0.7855
Epoch 6/10
3763/3763 [==============================] - 1s 308us/step - loss: 0.2861 - acc: 0.8786 - val_loss: 0.4798 - val_acc: 0.7982
Epoch 7/10
3763/3763 [==============================] - 1s 286us/step - loss: 0.2308 - acc: 0.9057 - val_loss: 0.4741 - val_acc: 0.8036
